In [ ]:
from constants.abs_paths import AbsDirPath
import pandas as pd

file_dir = AbsDirPath.KEYWORDS / "matched_wikis"

def load_all_csvs(dir):
    dfs = []
    for file_path in dir.glob("*.csv"):
        try:
            file = pd.read_csv(file_path)
            file['fname'] = file_path
            dfs.append(file)
            print(f"Loaded {file_path}")
        except:
            print(f"Error while loading {file_path}")

    df = pd.concat(dfs)
    return df


df = load_all_csvs(file_dir)

In [ ]:
file_dir = AbsDirPath.KEYWORDS / "matched_wikis_pq"

def load_all_csvs(dir):
    dfs = []
    for file_path in dir.glob("*.parquet"):
        try:
            file = pd.read_parquet(file_path)
            file['fname'] = file_path
            dfs.append(file)
            print(f"Loaded {file_path}")
        except:
            print(f"Error while loading {file_path}")

    df2 = pd.concat(dfs)
    return df2


df2 = load_all_csvs(file_dir)

In [ ]:
# from pandasgui import show
# show(df)

In [ ]:
df2.head()

In [ ]:
df2.pivot_table(index=["sentence", "keyword"], columns=["quality_attribute"], values="id", aggfunc="count", fill_value=0, margins=True)

In [ ]:
df2.pivot_table(index=["quality_attribute"], columns=["repo", "source"], values="id", aggfunc="count", fill_value=0, margins=True).sort_values(by="All", ascending=False)

In [ ]:
df2.pivot_table(index=["quality_attribute"], columns=["repo", "source"], values="id", aggfunc="count", fill_value=0, margins=True).sort_values(by="All", ascending=False)

In [ ]:
df2[(df2["repo"] == "scanpy") & (df2['quality_attribute'] == "energy efficiency")]

In [ ]:
df2.groupby(["sentence", "source", "keyword"])['id'].count().sort_values(ascending=False)

In [ ]:
df2['sentence_counts'] = df2.groupby(["sentence", "source", "keyword"])['id'].transform('count')

In [ ]:
df2[(df2['sentence_counts'] > 100) & ~(df2['repo'] == "root")].sort_values(by=['sentence_counts', "sentence", "source", "keyword"],ascending=[False]*4)

In [ ]:
import hashlib

quick_hash = lambda x: hashlib.md5(x.encode('utf-8')).hexdigest()

df2['text_hash'] = df2['sentence'].apply(quick_hash)

In [ ]:
df2['real_id'] = df2['repo'] + "." + df2['author'] + "." + df2['source'] + "." + df2['quality_attribute'] + "." +  df2['text_hash'] + "." + df2['keyword'].str.lower() + "." +df2['matched_word'].str.lower()

In [ ]:
df2.groupby(["real_id"])['id'].count().sort_values(ascending=False)

In [ ]:
df2.groupby(["real_id"])['id'].count().sort_values(ascending=False).count()

In [ ]:
df2['id'].count()

In [ ]:
942631 / 2375956

In [ ]:
df2['similar_count'] = df2.groupby(["real_id"])['id'].transform('count')

In [ ]:
df2 = df2.groupby(['real_id']).first()

In [ ]:
df2.head()

In [ ]:
df2[~df2['url'].str.startswith("https://root.cern/root/") & ~df2['url'].str.startswith("https://scanpy.readthedocs.io/en/stable/")]

In [ ]:
import itertools

sources = pd.Series(df2.source.unique()).str.split(".").str[0].tolist()
repos = df2[['repo', 'version', 'author']].drop_duplicates().values.tolist()
output_dir = AbsDirPath.OPTIMIZED_KEYWORDS
output_dir.mkdir(exist_ok=True)

for source, (repo, version, author) in itertools.product(sources, repos):
    output_file = output_dir / f"{author}.{repo}.{version}.{source}.parquet"
    df2[(df2['source'] == source) & (df2['repo'] == repo)].to_parquet(output_file, engine='pyarrow', compression='snappy', index=False)
    print(f"Saved {output_file}")

In [ ]:
df2